In [1]:
import requests
import base64

In [2]:
# Replace with your own Client ID and Client Secret
CLIENT_ID = "bdf151ffe6864c5eb2d60746096e712c"
CLIENT_SECRET = "29a73895f79d4f5b8aa530aef188876f"

In [3]:
# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [4]:
!pip install spotipy

In [5]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)
    
    # Initialize variables to handle pagination
    limit = 100  # Maximum tracks to retrieve per request
    offset = 0
    total_tracks = 1  # Initializing with 1 so that the loop starts
    music_data = []

    while offset < total_tracks:
        # Fetch tracks from the playlist with pagination
        playlist_tracks = sp.playlist_tracks(playlist_id, 
                                             fields='items(track(id, name, artists, album(id, name))), total',
                                             limit=limit, 
                                             offset=offset)

        # Update the total number of tracks to retrieve
        total_tracks = playlist_tracks['total']

        # Process each track
        for track_info in playlist_tracks['items']:
            track = track_info['track']
            track_name = track['name']
            artists = ', '.join([artist['name'] for artist in track['artists']])
            album_name = track['album']['name']
            album_id = track['album']['id']
            track_id = track['id']

            # Get audio features for the track
            audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

            # Get release date of the album
            try:
                album_info = sp.album(album_id) if album_id != 'Not available' else None
                release_date = album_info['release_date'] if album_info else None
            except:
                release_date = None

            # Get popularity of the track
            try:
                track_info = sp.track(track_id) if track_id != 'Not available' else None
                popularity = track_info['popularity'] if track_info else None
            except:
                popularity = None

            # Add additional track information to the track data
            track_data = {
                'Track Name': track_name,
                'Artists': artists,
                'Album Name': album_name,
                'Album ID': album_id,
                'Track ID': track_id,
                'Popularity': popularity,
                'Release Date': release_date,
                'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
                'Explicit': track_info.get('explicit', None),
                'External URLs': track_info.get('external_urls', {}).get('spotify', None),
                'Danceability': audio_features['danceability'] if audio_features else None,
                'Energy': audio_features['energy'] if audio_features else None,
                'Key': audio_features['key'] if audio_features else None,
                'Loudness': audio_features['loudness'] if audio_features else None,
                'Mode': audio_features['mode'] if audio_features else None,
                'Speechiness': audio_features['speechiness'] if audio_features else None,
                'Acousticness': audio_features['acousticness'] if audio_features else None,
                'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
                'Liveness': audio_features['liveness'] if audio_features else None,
                'Valence': audio_features['valence'] if audio_features else None,
                'Tempo': audio_features['tempo'] if audio_features else None,
                # Add more attributes as needed
            }

            music_data.append(track_data)

        # Increment the offset by the limit for the next batch of tracks
        offset += limit

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df


playlist_id = '3zWFJIpVFPkND5bkbKvsB4'
# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
print(music_df)


                         Track Name  \
0                     Chaila Chaila   
1                    Turn Back Time   
2                 Baitikochi Chuste   
3                     Gaali Vaaluga   
4                       Yenno Yenno   
..                              ...   
70             Dancin - Krono Remix   
71                            Beird   
72  Tum Hi Aana (From "Marjaavaan")   
73                     Ee Manchullo   
74                  Nemali Kulukula   

                                            Artists  \
0                                   Chiranjeevi, KK   
1                                     Daniel Schulz   
2                               Anirudh Ravichander   
3                               Anirudh Ravichander   
4                                 Karthik, Chinmayi   
..                                              ...   
70                        Aaron Smith, Luvli, Krono   
71                                     Jack Stauber   
72         Payal Dev, Jubin Nautiyal

In [15]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

While providing music recommendations to users, it is important to recommend the latest releases. For this, we need to give more weight to the latest releases in the recommendations. Let’s write a function to solve this problem:

In [17]:
#Funtion to calculate the weighted popularity
def calculate_weighted_popularity(release_date):
    #convert the release date into datetime format
    release_date = datetime.strptime(release_date, '%Y-%m-%d')
    
    #Calculate the time span between release date ad today
    
    time_span = datetime.now() - release_date
    
    #calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1/ (time_span.days + 1)
    return weight

In [18]:
# Nomalizing the music features

scaler = MinMaxScaler()
music_features = music_df[['Danceability','Energy','Key',
                          'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [19]:
music_features_scaled

array([[9.06757223e-01, 6.67335516e-01, 1.81818182e-01, 7.76386342e-01,
        0.00000000e+00, 4.49115686e-01, 4.85137640e-01, 2.72335845e-05,
        5.45590433e-02, 6.72201855e-01, 2.62684520e-01],
       [6.91493036e-01, 3.33614954e-01, 6.36363636e-01, 8.38689591e-01,
        1.00000000e+00, 1.76862859e-02, 2.47013798e-01, 0.00000000e+00,
        3.30343797e-01, 2.13931421e-01, 2.29966927e-01],
       [5.85587660e-01, 4.36054494e-01, 5.45454545e-01, 8.50313073e-01,
        0.00000000e+00, 1.31342418e-01, 6.24579529e-01, 0.00000000e+00,
        1.44992526e-01, 6.64653871e-01, 1.79930628e-01],
       [6.88039599e-01, 8.23635020e-01, 0.00000000e+00, 8.50486041e-01,
        0.00000000e+00, 3.97216585e-02, 6.88937324e-01, 3.22927879e-06,
        1.92825112e-01, 6.34461937e-01, 3.43639590e-01],
       [8.05456429e-01, 4.12820784e-01, 9.09090909e-02, 7.40235929e-01,
        0.00000000e+00, 3.01536677e-02, 7.50077229e-01, 0.00000000e+00,
        4.24514200e-01, 5.84860901e-01, 2.63144309e-

###  Building a recommendation system

We will create a hybrid recommendation system for music recommendations. The first approach will be based on recommending music based on music audio features, and the second approach will be based on recommending music based on weighted popularity.

In [20]:
# music based on music audio features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [21]:
calculate_weighted_popularity(
        music_df.loc[music_df['Track Name'] == "Turn Back Time", 'Release Date'].values[0]
    )

0.0003996802557953637

Now the Function to generate music recommendations based on weighted popularity and combine it with the recommendations of the content-based filtering method using the hybrid approach:

In [22]:
import pandas as pd

def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    weighted_popularity_score = popularity_score * calculate_weighted_popularity(
        music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]
    )

    new_entry = pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })

    hybrid_recommendations = pd.concat([content_based_rec, new_entry], ignore_index=True)

    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]

    return hybrid_recommendations

In [33]:
input_song_name = "Karige Loga"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'Karige Loga':
        Track Name                                          Artists  \
0   Oh Priya Priya                         Adnan Sami, Nithya Menon   
1       Dance Like                             Harrdy Sandhu, Jaani   
2   Varinche Prema                                       Haricharan   
3         Tu Tu Tu                  Kunal Ganjawala, Shreya Ghoshal   
4  Nemali Kulukula  Hariharan, Unnikrishnan, Shweta Mohan, Chinmayi   

                  Album Name Release Date  Popularity  
0                       Ishq   2012-02-03        57.0  
1                 Dance Like   2019-11-30        56.0  
2  Malli Malli Idi Rani Roju   2014-12-07        48.0  
3                      Ready   2008-05-19        47.0  
4                     Rangam   2011-04-09        44.0  
